In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-credit-default-risk/sample_submission.csv
/kaggle/input/home-credit-default-risk/bureau_balance.csv
/kaggle/input/home-credit-default-risk/POS_CASH_balance.csv
/kaggle/input/home-credit-default-risk/application_train.csv
/kaggle/input/home-credit-default-risk/HomeCredit_columns_description.csv
/kaggle/input/home-credit-default-risk/application_test.csv
/kaggle/input/home-credit-default-risk/previous_application.csv
/kaggle/input/home-credit-default-risk/credit_card_balance.csv
/kaggle/input/home-credit-default-risk/installments_payments.csv
/kaggle/input/home-credit-default-risk/bureau.csv


In [2]:
import pandas as pd
import numpy as np
import re
import pickle
import gc

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

import lightgbm as lgb

import warnings
warnings.filterwarnings("ignore")

In [3]:
application_train = pd.read_csv("/kaggle/input/home-credit-default-risk/application_train.csv")
print(application_train.shape)
application_train.head()

(307511, 122)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [5]:
application_train = reduce_mem_usage(application_train)

Memory usage of dataframe is 286.23 MB
Memory usage after optimization is: 59.54 MB
Decreased by 79.2%


In [9]:
X_train = application_train.drop(columns = ["TARGET", "SK_ID_CURR"])
y_train = application_train["TARGET"]
id_train = application_train[["SK_ID_CURR"]]

In [13]:
for col in X_train.columns:
    if X_train[col].dtype == "0":
        X_train[col] = X_train[col].astype("category")

In [15]:
print(f"mean : {y_train.mean()}")
y_train.value_counts()

mean : 0.08072881945686496


TARGET
0    282686
1     24825
Name: count, dtype: int64

In [17]:
cv = list(StratifiedKFold(n_splits = 5, shuffle = True, random_state = 123).split(X_train, y_train))
print("index(train) : ", cv[0][0])
print("index(valid) : ", cv[0][1])

index(train) :  [     0      1      3 ... 307508 307509 307510]
index(valid) :  [     2     11     22 ... 307488 307495 307497]


In [18]:
nfold = 0
idx_tr, idx_va = cv[nfold][0], cv[nfold][1]

X_tr, y_tr, id_tr = X_train.loc[idx_tr, :], y_train[idx_tr], id_train.loc[idx_tr, :]
X_va, y_va, id_va = X_train.loc[idx_va, :], y_train[idx_va], id_train.loc[idx_va, :]
print(X_tr.shape, y_tr.shape, id_tr.shape)
print(X_va.shape, y_va.shape, id_va.shape)

(246008, 120) (246008,) (246008, 1)
(61503, 120) (61503,) (61503, 1)


In [22]:
params = {
    "boosting_type":"gbdt",
    "objective":"binary",
    "metric":"auc",
    "learning_rate":0.05,
    "num_leaves":32,
    "n_estimators":100000,
    "random_state":123,
    "importance_type":"gain"
}

model = lgb.LGBMClassifier(**params)
model.fit(X_tr,
         y_tr,
         eval_set = [(X_tr, y_tr), (X_va, y_va)],
          early_stopping_rounds = 100,
          verbose = 100
         )

with open("model_lgb_fold0.pickle", "wb") as f:
    pickle.dump(model, f, protocol = 4)

[100]	training's auc: 0.782506	valid_1's auc: 0.755903
[200]	training's auc: 0.808961	valid_1's auc: 0.758356
[300]	training's auc: 0.829245	valid_1's auc: 0.757774


In [23]:
y_tr_pred = model.predict_proba(X_tr)[:, 1]
metric_tr = roc_auc_score(y_tr, y_tr_pred)

y_va_pred = model.predict_proba(X_va)[:, 1]
metric_va = roc_auc_score(y_va, y_va_pred)

metrics = []

metrics.append([nfold, metric_tr, metric_va])

print(f"[auc] tr:{metric_tr:.4f}, va:{metric_va:.4f}")

[auc] tr:0.8126, va:0.7586


In [24]:
train_oof = np.zeros(len(X_train))
train_oof[idx_va] = y_va_pred

In [25]:
imp_fold = pd.DataFrame({"col":X_train.columns, "imp":model.feature_importances_, "nfold":nfold})
display(imp_fold.sort_values("imp", ascending = False)[:10])

imp = pd.DataFrame()
imp = pd.concat([imp, imp_fold])

,col,imp,nfold
41,EXT_SOURCE_3,66225.020483,0
40,EXT_SOURCE_2,52568.833805,0
38,ORGANIZATION_TYPE,20218.523523,0
39,EXT_SOURCE_1,19776.252288,0
6,AMT_CREDIT,8111.321247,0
8,AMT_GOODS_PRICE,7120.960365,0
15,DAYS_BIRTH,7042.223005,0
7,AMT_ANNUITY,6992.551795,0
16,DAYS_EMPLOYED,5236.514120,0
26,OCCUPATION_TYPE,4376.651746,0


In [28]:
metrics = np.array(metrics)
print(metrics)

print(f"[cv] tr:{metrics[:, 1].mean():.4f}+-{metrics[:, 1].std():.4f}, va:{metrics[:, 2].mean():.4f}+-{metrics[:, 2].std():.4f}")

print(f"[oof] {roc_auc_score(y_train, train_oof):.4f}")

[[0.         0.81257796 0.75859528]]
[cv] tr:0.8126+-0.0000, va:0.7586+-0.0000
[oof] 0.5103


In [30]:
train_oof = pd.concat([
    id_train,
    pd.DataFrame({"true":y_train, "pred":train_oof})
],axis = 1)
train_oof.head()

,SK_ID_CURR,true,pred
0,100002,1,0.000000
1,100003,0,0.000000
2,100004,0,0.031866
3,100006,0,0.000000
4,100007,0,0.000000


In [31]:
imp = imp.groupby("col")["imp"].agg(["mean", "std"]).reset_index()
imp.columns = ["col", "imp", "imp_std"]
imp.head()

,col,imp,imp_std
0,AMT_ANNUITY,6992.551795,NaN
1,AMT_CREDIT,8111.321247,NaN
2,AMT_GOODS_PRICE,7120.960365,NaN
3,AMT_INCOME_TOTAL,1595.740609,NaN
4,AMT_REQ_CREDIT_BUREAU_DAY,128.842901,NaN


In [32]:
cv

[(array([     0,      1,      3, ..., 307508, 307509, 307510]),
  array([     2,     11,     22, ..., 307488, 307495, 307497])),
 (array([     0,      1,      2, ..., 307508, 307509, 307510]),
  array([     3,      6,      8, ..., 307492, 307501, 307504])),
 (array([     2,      3,      4, ..., 307507, 307509, 307510]),
  array([     0,      1,     12, ..., 307490, 307498, 307508])),
 (array([     0,      1,      2, ..., 307508, 307509, 307510]),
  array([     5,     14,     16, ..., 307503, 307506, 307507])),
 (array([     0,      1,      2, ..., 307506, 307507, 307508]),
  array([     4,      7,      9, ..., 307505, 307509, 307510]))]

In [33]:
def train_lgb(input_x,
             input_y,
             input_id,
             params,
             list_nfold = [0,1,2,3,4],
             n_splits = 5):
    
    train_oof = np.zeros(len(input_x))
    metrics = []
    imp = pd.DataFrame()
    
    # cross validation
    cv = list(StratifiedKFold(n_splits = n_splits, shuffle = True, random_state = 123).split(input_x, input_y))
    
    for nfold in list_nfold:
        print("-"*20,nfold,"-"*20)
        
        # make datasets
        idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
        x_tr, y_tr, id_tr = input_x.loc[idx_tr, :], input_y.loc[idx_tr], input_id.loc[idx_tr]
        x_va, y_va, id_va = input_x.loc[idx_va, :], input_y.loc[idx_va], input_id.loc[idx_va]
        print(x_tr.shape, x_va.shape)
        
        # train
        model = lgb.LGBMClassifier(**params)
        model.fit(x_tr,
                 y_tr,
                 eval_set = [(x_tr, y_tr), (x_va, y_va)],
                 early_stopping_rounds = 100,
                 verbose = 100
                 )
        fname_lgb = f"model_lgb_fold{nfold}.pickle"
        with open(fname_lgb, "wb") as f:
            pickle.dump(model, f, protocol = 4)
        
        # evaluate
        y_tr_pred = model.predict_proba(x_tr)[:,1]
        y_va_pred = model.predict_proba(x_va)[:,1]
        metric_tr = roc_auc_score(y_va, y_va_pred)
        metrics.append([nfold, metric_tr, metric_va])
        print(f"[auc] tr:{metric_tr:.4f}, va:{metric_va:.4f}")
        
        #oof
        train_oof[idx_va] = y_va_pred
        
        # imp
        _imp = pd.DataFrame({"col":input_x.columns, "imp":model.feature_importances_, "nfold":nfold})
        imp = pd.concat([imp, _imp])
    
    print("-"*20, "result", "-"*20)
    # metric
    metrics = np.array(metrics)
    print(metrics)
    print(f"[cv] tr:{metrics[:, 1].mean():.4f}+-{metrics[:, 1].std():.4f}, va:{metrics[:, 2].mean():.4f}+-{metrics[:, 2].std():.4f}")
    print(f"[oof] {roc_auc_score(input_y, train_oof)}")
    
    # oof
    train_oof = pd.concat([input_id, pd.DataFrame({"pred":train_oof})], axis = 1)
    
    # importance
    imp = imp.groupby("col")["imp"].agg(["mean", "std"]).reset_index()
    imp.columns = ["col", "imp", "imp_std"]
    
    return train_oof, imp, metrics
            
    
        
        

In [35]:
# ハイパーパラメータの設定
params = {
    "boosting_type":"gbdt",
    "objective":"binary",
    "metric":"auc",
    "learning_rate":0.05,
    "num_leaves":32,
    "n_estimators":100000,
    "random_state":123,
    "importance_type":"gain"
}

# 学習の実行
train_oof, imp, metrics = train_lgb(X_train,
                                   y_train,
                                   id_train,
                                   params,
                                   list_nfold = [0,1,2,3,4],
                                   n_splits = 5)

-------------------- 0 --------------------
(246008, 120) (61503, 120)
[100]	training's auc: 0.782506	valid_1's auc: 0.755903
[200]	training's auc: 0.808961	valid_1's auc: 0.758356
[300]	training's auc: 0.829245	valid_1's auc: 0.757774
[auc] tr:0.7586, va:0.7586
-------------------- 1 --------------------
(246009, 120) (61502, 120)
[100]	training's auc: 0.782531	valid_1's auc: 0.756239
[200]	training's auc: 0.808862	valid_1's auc: 0.758924
[300]	training's auc: 0.829564	valid_1's auc: 0.758779
[auc] tr:0.7590, va:0.7586
-------------------- 2 --------------------
(246009, 120) (61502, 120)
[100]	training's auc: 0.782101	valid_1's auc: 0.758221
[200]	training's auc: 0.809587	valid_1's auc: 0.760104
[300]	training's auc: 0.830474	valid_1's auc: 0.760275
[400]	training's auc: 0.847388	valid_1's auc: 0.759875
[auc] tr:0.7604, va:0.7586
-------------------- 3 --------------------
(246009, 120) (61502, 120)
[100]	training's auc: 0.783853	valid_1's auc: 0.754567
[200]	training's auc: 0.811501

In [37]:
imp.sort_values("imp", ascending = False)[:10]

,col,imp,imp_std
38,EXT_SOURCE_3,65353.907478,1558.201212
37,EXT_SOURCE_2,54545.388309,1251.798934
102,ORGANIZATION_TYPE,21441.917474,1450.246190
36,EXT_SOURCE_1,20051.934248,685.852224
1,AMT_CREDIT,8263.228728,410.384434
22,DAYS_BIRTH,7645.589110,689.458833
2,AMT_GOODS_PRICE,7263.054566,405.837031
0,AMT_ANNUITY,6762.953640,479.302045
23,DAYS_EMPLOYED,5810.288375,552.937730
101,OCCUPATION_TYPE,5502.675859,831.872392


In [41]:
# 推論用データの作成
application_test = pd.read_csv("/kaggle/input/home-credit-default-risk/application_test.csv")
application_test = reduce_mem_usage(application_test)

X_test = application_test.drop(columns = ["SK_ID_CURR"])
id_test = application_test[["SK_ID_CURR"]]

for col in X_test.columns:
    if X_test[col].dtype == "0":
        X_test[col] = X_test[col].astype("category")

Memory usage of dataframe is 45.00 MB
Memory usage after optimization is: 9.40 MB
Decreased by 79.1%


In [42]:
def predict_lgb(input_x, input_id, list_nfold = [0,1,2,3,4]):
    pred = np.zeros((len(input_x), len(list_nfold)))
    for nfold in list_nfold:
        print("-"*20, nfold, "-"*20)
        fname_lgb = f"model_lgb_fold{nfold}.pickle"
        with open(fname_lgb, "rb") as f:
            model = pickle.load(f)
        pred[:, nfold] = model.predict_proba(input_x)[:,1]
    
    pred = pd.concat([
        input_id,
        pd.DataFrame({"pred":pred.mean(axis = 1)})
    ], axis = 1)
    
    print("Done.")
    
    return pred

In [43]:
test_pred = predict_lgb(X_test, id_test, list_nfold = [0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [44]:
df_submit = test_pred.rename(columns = {"pred":"TARGET"})
print(df_submit.shape)
print(df_submit.head())

# ファイル出力
df_submit.to_csv("submission_baseline.csv", index = None)

(48744, 2)
   SK_ID_CURR    TARGET
0      100001  0.041810
1      100005  0.126400
2      100013  0.022495
3      100028  0.039680
4      100038  0.156628
